In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow as imshow
import math

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

#!mkdir /content/images/
#!unzip /content/drive/MyDrive/Colab\ Notebooks/data_fullsize.zip -d /content/images/

In [ ]:
def erode(size, bin_img):
  kernel = np.ones((size, size), np.uint8)
  return cv2.erode(bin_img, kernel)

def dilate(size, bin_img):
  kernel = np.ones((size, size), np.uint8)
  return cv2.dilate(bin_img, kernel)

def open(size, bin_img):
  return dilate(size, erode(size, bin_img))

def close(size, bin_img):
  return erode(size, dilate(size, bin_img))

In [ ]:
from skimage.segmentation import flood, flood_fill

def get_box_of_bin(bin_image):
  xmin = np.inf
  xmax = -np.inf
  ymin = np.inf
  ymax = -np.inf
  
  for idy, y in enumerate(bin_image):
    for idx, x in enumerate(y):
      if(x == 255):
        xmin = (idx if idx<xmin else xmin)
        xmax = (idx if idx>xmax else xmax)
        ymin = (idy if idy<ymin else ymin)
        ymax = (idy if idy>ymax else ymax)
  return xmin, ymin, xmax, ymax

def fill(bin_image):
  for idy, y in enumerate(bin_image):
    for idx, x in enumerate(y):
      if(x == 255):
        flooded = flood_fill(bin_image, (idy, idx), 0)
        xmin, ymin, xmax, ymax = get_box_of_bin(bin_image - flooded)
        diff = bin_image-flooded
        return flooded, [xmin, ymin, xmax, ymax]
  return 0, []

def get_white_boxes(bin_img):
  filler = bin_img
  boxes = []
  while not isinstance(filler, int):
    flooded, box = fill(filler)
    filler = flooded
    if(len(box)==4 and box[0]<box[2] and box[1]<box[3]):
      boxes.append(box)

  return boxes

In [ ]:
def seperate_bin_areas(seperator, size, bin_img, i):
  boxes = get_white_boxes(bin_img)
  dice_count = len(boxes)
  if dice_count == 5 or i >= 5:
    return boxes
  elif dice_count > 5:
    return seperate_bin_areas(close, np.ceil(size*1.25), bin_img, i+1)
  else:
    s = np.floor(size*1.25)
    return seperate_bin_areas(open, (2 if s<2 else s), bin_img, i+1)

In [ ]:
import os
!mkdir /content/singledice/
basedir = r"/content/images/simple"
for file in os.listdir(basedir):
    if file.endswith(".jpg"):
        filename = os.path.join(basedir, file)
        filenoend = file.split(".")[0]

        src = cv2.imread(filename)   
        #print(src.shape)  
        src = cv2.resize(src, (256,256))
        original = src.copy()
        imshow(cv2.resize(src, (256,256)))

        gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

        (thresh, bin) = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        
        boxes = open(2, bin)#seperate_bin_areas(open, 2, bin, 0)
        
        for box in boxes:
            cv2.rectangle(src,(box[0],box[1]),(box[2],box[3]),(200,0,0),2) #Draw rectangle boundingbox
        imshow(cv2.resize(src, (256,256)))
        
        print(len(boxes))
        for idx, box in enumerate(boxes):
              im = original[box[1]:box[3],box[0]:box[2]]
              imshow(cv2.resize(im, (64,64)))
              if(len(boxes)==5):
                cv2.imwrite("/content/singledice/"+str(idx) + "_" + str(filenoend) + ".jpg", im)

CV2 Find Contours - If opening and closing is not seperating wel

In [ ]:
#!zip -r /content/singledice.zip /content/singledice

#from google.colab import files
#files.download("/content/singledice.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# import the necessary packages
import numpy as np
# Malisiewicz et al.
def non_max_suppression_fast(boxes, overlapThresh):
	# if there are no boxes, return an empty list
	if len(boxes) == 0:
		return []
	# if the bounding boxes integers, convert them to floats --
	# this is important since we'll be doing a bunch of divisions
	if boxes.dtype.kind == "i":
		boxes = boxes.astype("float")
	# initialize the list of picked indexes	
	pick = []
	# grab the coordinates of the bounding boxes
	x1 = boxes[:,0]
	y1 = boxes[:,1]
	x2 = boxes[:,2]
	y2 = boxes[:,3]
	# compute the area of the bounding boxes and sort the bounding
	# boxes by the bottom-right y-coordinate of the bounding box
	area = (x2 - x1 + 1) * (y2 - y1 + 1)
	idxs = np.argsort(y2)
	#print(x1, y1, x2, y2)
	# keep looping while some indexes still remain in the indexes
	# list
	while len(idxs) > 0:
		# grab the last index in the indexes list and add the
		# index value to the list of picked indexes
		last = len(idxs) - 1
		i = idxs[last]
		pick.append(i)
		# find the largest (x, y) coordinates for the start of
		# the bounding box and the smallest (x, y) coordinates
		# for the end of the bounding box
		xx1 = np.maximum(x1[i], x1[idxs[:last]])
		yy1 = np.maximum(y1[i], y1[idxs[:last]])
		xx2 = np.minimum(x2[i], x2[idxs[:last]])
		yy2 = np.minimum(y2[i], y2[idxs[:last]])
		# compute the width and height of the bounding box
		w = np.maximum(0, xx2 - xx1 + 1)
		h = np.maximum(0, yy2 - yy1 + 1)
		# compute the ratio of overlap
		overlap = (w * h) / area[idxs[:last]]
		# delete all indexes from the index list that have
		idxs = np.delete(idxs, np.concatenate(([last],
			np.where(overlap > overlapThresh)[0])))
	# return only the bounding boxes that were picked using the
	# integer data type
	return boxes[pick].astype("int")

In [ ]:
def roi_by_contours(base_img):
  contours, hierarchy = cv2.findContours(base_img,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
  i = 0
  boxes = []
  for cnt in contours:
      i += 1
      x,y,w,h = cv2.boundingRect(cnt)
      roi = src[y:y+h,x:x+w]
      boxes.append([x,y,x+w,y+h])
        #imshow(cv2.resize(roi, (128,128)))
      #cv2.imwrite("/content/"+str(idx) + '.jpg', roi) #Persist roi
      #cv2.rectangle(im,(x,y),(x+w,y+h),(200,0,0),2) #Draw rectangle boundingbox
  return boxes

In [ ]:
def get_boxes_by_contours(src_img):
  gray = cv2.cvtColor(src_img, cv2.COLOR_BGR2GRAY)

  (thresh, bin) = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

  canny_bin = cv2.Canny(bin, 50, 200, None, 3)

  boxes = roi(canny_bin)

  one_eye = cv2.imread(r"/content/ref.png")
  #boxes = remove_unwanted(boxes, one_eye, thresh=0.8)

  boxes = remove_duplicates(boxes)
  boxes = remove_inlayer(boxes)

  return boxes

In [ ]:
#remove exact duplicates
def remove_duplicates(boxes):
  duplicates = 0
  for idx1, box1 in enumerate(boxes):
    for idx2, box2 in enumerate(boxes):
      if(idx1 != idx2 and box1[0] == box2[0] and box1[1] == box2[1] and box1[2] == box2[2] and box1[3] == box2[3]):
        boxes.remove(box2)
        duplicates += 1
  return boxes
  #print("removed "+str(duplicates)+" duplicate pictures")
  
#Remove real Containings
def remove_inlayer(boxes):
  found = True
  while found:
    found = False
    for box1 in boxes:
      for box2 in boxes:
        #box1.x1 < box2.x1 and box1.x2 > box2.x2  
        #box1.y1 < box2.y1 and box1.y2 > box2.y2
          if(box1!=box2 and box1[0]<=box2[0] and box1[2]>=box2[2] and box1[1]<=box2[1] and box1[3]>=box2[3]):
            boxes.remove(box2)
            found = True
  return boxes

In [ ]:
import os

basedir = r"/content/images/simple"
for file in os.listdir(basedir):
    if file.endswith(".jpg"):
        filename = os.path.join(basedir, file)
        filenoend = filename.split(".")[0]

        src = cv2.imread(filename)   
        original = src.copy()
        imshow(cv2.resize(src, (256,256)))

        boxes = get_boxes_by_contours(src)
        if(len(boxes)>5):
          boxes = non_max_suppression_fast(np.array(boxes), 0.8)
        
        print(len(boxes))
        for idx, box in enumerate(boxes):
              im = src[box[1]:box[3],box[0]:box[2]]
              imshow(cv2.resize(im, (64,64)))
              #cv2.imwrite("/content/rois/"+str(idx) + "_" + str(filenoend) + ".jpg", im)

AttributeError: ignored

In [ ]:
import numpy as np
boxes = np.array(boxes)
new_boxes = non_max_suppression_fast(boxes, 0.1)
print(len(new_boxes))
print(len(boxes))

5
5


In [ ]:
for box in boxes:
  imshow(src[box[1]:box[3],box[0]:box[2]])

In [ ]:
src = cv2.imread("/content/images/simple/IMG_20211121_215151.jpg")   
print(src.shape)
src = cv2.resize(src, (256,256))
original = src.copy()
imshow(cv2.resize(src, (256,256)))

gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

(thresh, bin) = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
imshow(bin)
imshow(open(2, bin))
boxes = seperate_bin_areas(open, 2, bin, 0)

for box in boxes:
    cv2.rectangle(src,(box[0],box[1]),(box[2],box[3]),(200,0,0),2) #Draw rectangle boundingbox
imshow(cv2.resize(src, (256,256)))

print(len(boxes))
for idx, box in enumerate(boxes):
      im = original[box[1]:box[3],box[0]:box[2]]
      imshow(cv2.resize(im, (64,64)))

# IoU

In [ ]:
# import the necessary packages
import numpy as np
import cv2

# Calculate IoU for every Boundingbox for every GroundTruth; 5x5 
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = (xB - xA) * (yB - yA)

    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou